In [54]:
import pandas as pd
dataset = pd.read_csv('data.csv')

In [58]:
dataset = pd.read_csv('data.csv', parse_dates=["Date"])
dataset.columns = ["Date", "Start Date", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "Box Score", "OT?", "Attend", "Notes"]

In [59]:
dataset["HomeWin"] = dataset["VisitorPts"] <dataset["HomePts"]
y_true = dataset["HomeWin"].values

In [62]:
dataset["HomeLastWin"] = False
dataset["VisitorLastWin"] = False
from _collections import defaultdict
won_last = defaultdict(int)
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    dataset.loc[index] = row
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]

In [63]:
from sklearn.tree import  DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
clf = DecisionTreeClassifier(random_state=14)
x_previouswins = dataset[["HomeLastWin","VisitorLastWin"]].values
print(x_previouswins)
score = cross_val_score(clf, x_previouswins, y_true, scoring='accuracy')
print("{0:.1f}".format(np.mean(score)*100))



[[0 0]
 [0 0]
 [0 0]
 ...
 [False False]
 [True True]
 [True False]]
60.3


In [74]:
last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0
for index,row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team,visitor_team]))
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    dataset.loc[index] = row
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner

In [79]:
x_lastwinner = dataset[["HomeLastWin","VisitorLastWin","HomeTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
score = cross_val_score(clf, x_lastwinner, y_true, scoring="accuracy")
print("{0:.1f}".format(np.mean(score)*100))



59.6


In [80]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit(dataset["Home Team"].values)
home_team = encoding.transform(dataset["Home Team"].values)
visitor_team = encoding.transform(dataset["Visitor Team"].values)
x_team = np.vstack([home_team,visitor_team]).T



In [81]:
from sklearn.preprocessing import  OneHotEncoder
onehot = OneHotEncoder()
x_team_expanded = onehot.fit_transform(x_team).todense()
clf = DecisionTreeClassifier(random_state=14)
score = cross_val_score(clf, x_team_expanded, y_true, scoring='accuracy')
print("{0:.1f}".format(np.mean(score)*100))


58.3
